# Chapter 6: Maintaining your Delta Lake
> The following exercises use the New York Times [Covid-19 NYT Dataset](https://github.com/delta-io/delta-docs/tree/main/static/quickstart_docker/rs/data/COVID-19_NYT).

The dataset can be found in the `delta_quickstart` docker.

In [4]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, desc, to_date
from delta.tables import DeltaTable

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS default.covid_nyt (
  date DATE
) USING DELTA
TBLPROPERTIES('delta.logRetentionDuration'='interval 7 days');
""")


In [ ]:
spark.sql("show tables").show()

In [ ]:
# will be empty on the first run. this is expected
len(spark.table("default.covid_nyt").inputFiles())

In [ ]:
# uncomment if you'd like to begin again
#spark.sql("drop table default.covid_nyt")

## Start Populating the Table
> The next three commands are used to show Schema Evolution and Validation with Delta Lake

In [ ]:
# Populate the Table reading the Parquet covid_nyc Data
# note: this will fail on the first run, and that is okay
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .saveAsTable("default.covid_nyt"))

In [ ]:
# one step closer, there is still something missing...
# and yes, this operation still fails... if only...
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .mode("append")
      .saveAsTable("default.covid_nyt"))

## Schema Evolution: Handle Automatically
If you trust the upstream data source (provider) then you can add the `option("mergeSchema", "true")`. Otherwise, it is better to specifically select a subset of the columns you expected to see. In this example use case, the only known column is `date`, so it is fairly safe to power ahead.

In [ ]:
# Evolve the Schema. (Showcases how to auto-merge changes to the schema)
# note: if you can trust the upstream, then this option is perfectly fine
# however, if you don't trust the upstream, then it is good to opt-in to the 
# changing columns.

(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .mode("append")
      .option("mergeSchema", "true")
      .saveAsTable("default.covid_nyt")
    )

In [ ]:
df = spark.table("default.covid_nyt")
df.count()

# Alternatives to Auto Schema Evolution
In the previous case, we used `.option("mergeSchema", "true")` to modify the behavior of the Delta Lake writer. While this option simplifies how we evolve our Delta Lake table schemas, it comes at the price of not being fully aware of the changes to our table schema. In the case where there are unknown columns being introduced from an upstream source, you'll want to know which columns are intended to bring forward, and which columns can be safely ignored.

## Intentionally Adding Columns with Alter Table

In [ ]:
# manually set the columns. This is an example of intentional opt-in to the new columns outside of '.option("mergeSchema", "true")`. 
# Note: this can be run once, afterwards the ADD columns will fail since they already exist
spark.sql("""
ALTER TABLE default.covid_nyt 
ADD columns (
  county STRING,
  state STRING,
  fips INT,
  cases INT,
  deaths INT
);
""")
# notice how we are only using `.mode("append")` and explicitly add `.option("mergeSchema", "false")`. 
# this is how we stop unwanted columns from being freely added to our Delta Lake tables. It comes at the cost of raising exceptions and failing the job.
# a failed job might seem like a bad option, but it is the cheaper option since you are intentionally blocking unknown data from flowing into your tables. 
(spark.read
      .format("parquet")
      .load("/opt/spark/work-dir/rs/data/COVID-19_NYT/*.parquet")
      .withColumn("date", to_date("date", "yyyy-MM-dd"))
      .write
      .format("delta")
      .option("mergeSchema", "false")
      .mode("append")
      .saveAsTable("default.covid_nyt"))

In [ ]:
spark.sql("describe extended default.covid_nyt").show(truncate=False)

In [79]:
spark.sql("select * from default.covid_nyt limit 10").show(truncate=True)

+----------+----------+--------+-----+-----+------+
|      date|    county|   state| fips|cases|deaths|
+----------+----------+--------+-----+-----+------+
|2020-05-19|  Lawrence|Illinois|17101|    4|     0|
|2020-05-19|       Lee|Illinois|17103|   75|     1|
|2020-05-19|Livingston|Illinois|17105|   27|     1|
|2020-05-19|     Logan|Illinois|17107|   10|     0|
|2020-05-19|     Macon|Illinois|17115|  170|    17|
|2020-05-19|  Macoupin|Illinois|17117|   42|     1|
|2020-05-19|   Madison|Illinois|17119|  499|    45|
|2020-05-19|    Marion|Illinois|17121|   49|     0|
|2020-05-19|  Marshall|Illinois|17123|    5|     0|
|2020-05-19|     Mason|Illinois|17125|   16|     0|
+----------+----------+--------+-----+-----+------+



# Adding and Modifying Table Properties

In [93]:
spark.sql("""
  ALTER TABLE default.covid_nyt 
  SET TBLPROPERTIES (
    'catalog.team_name'='dldg_authors',
    'catalog.engineering.comms.slack'='https://delta-users.slack.com/archives/CG9LR6LN4',
    'catalog.engineering.comms.email'='dldg_authors@gmail.com',
    'catalog.table.classification'='all-access'
  )""")


DataFrame[]

In [ ]:
# view the table history
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, 'default.covid_nyt')
dt.history(10).select("version", "timestamp", "operation").show()

In [ ]:
# use DeltaTable to view
dt.detail().select("properties").show(truncate=False)

In [ ]:
# view the table properties
spark.sql("show tblproperties default.covid_nyt").show(truncate=False)

## Removing Table Properties

In [ ]:
# add incorrect table property
# which is blocked by default
spark.conf.set("spark.databricks.delta.allowArbitraryProperties.enabled","true")
# now we can make a mistake
spark.sql("""
  ALTER TABLE default.covid_nyt 
  SET TBLPROPERTIES (
    'delta.loRgetentionDuratio'='interval 7 days'
  )""")

In [98]:
# luckily, we can remove the unwanted table property using UNSET
spark.sql("""
  ALTER TABLE default.covid_nyt 
  UNSET TBLPROPERTIES ('delta.loRgetentionDuratio')
""")
# now that we are done, let's just add back the safe guard again
spark.conf.set("spark.databricks.delta.allowArbitraryProperties.enabled","false")

## Delta Table Optimization

In [ ]:
## Creating the Small File Problem

from delta.tables import DeltaTable
(DeltaTable.createIfNotExists(spark)
    .tableName("default.nonoptimal_covid_nyt")
    .property("description", "table to be optimized")
    .property("catalog.team_name", "dldg_authors")
    .property("catalog.engineering.comms.slack",
	"https://delta-users.slack.com/archives/CG9LR6LN4")
    .property("catalog.engineering.comms.email","dldg_authors@gmail.com")
    .property("catalog.table.classification","all-access")
    .addColumn("date", "DATE")
    .addColumn("county", "STRING")
    .addColumn("state", "STRING")
    .addColumn("fips", "INT")
    .addColumn("cases", "INT")
    .addColumn("deaths", "INT")
    .execute())

In [ ]:
#spark.sql("drop table default.nonoptimal_covid_nyt")

In [ ]:
# you can remove `repartition(9000)` and add write...option('maxRecordsPerFile`, 10000)
# to generate more files using the DataFrameWriter
(spark
   .table("default.covid_nyt")
   .repartition(9000)
   .write
   .format("delta")
   .mode("overwrite")
   #.option("maxRecordsPerFile", 1000)
   .saveAsTable("default.nonoptimal_covid_nyt")
)

## Using Optimize to Fix the Small Files Problem

In [5]:
# set the maxFileSize to a bin-size for optimize
spark.conf.set("spark.databricks.delta.optimize.maxFileSize", 1024*1024*1024)
(
    DeltaTable.forName(spark, "default.nonoptimal_covid_nyt")
    .optimize()
    .executeCompaction()
)

23/06/08 23:57:55 WARN TaskSetManager: Stage 24 contains a task of very large size (1584 KiB). The maximum recommended task size is 1000 KiB.


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [6]:
# Viewing the results of Optimize
from pyspark.sql.functions import col
(
    DeltaTable.forName(spark, "default.nonoptimal_covid_nyt")
    .history(10)
    .where(col("operation") == "OPTIMIZE")
    .select("version", "timestamp", "operation", "operationMetrics.numRemovedFiles", "operationMetrics.numAddedFiles")
    .show(truncate=False)
)


+-------+-----------------------+---------+---------------+-------------+
|version|timestamp              |operation|numRemovedFiles|numAddedFiles|
+-------+-----------------------+---------+---------------+-------------+
|3      |2023-06-08 23:58:34.667|OPTIMIZE |9000           |1            |
+-------+-----------------------+---------+---------------+-------------+



In [ ]:
# rewind and try again
# note: the table version of the OPTIMIZE operation needs to be referenced to take the prior version
#(DeltaTable.forName(spark, "default.nonoptimal_covid_nyt").restoreToVersion(1))

## Partitioning, Repartitioning, and Default Partitions

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.types import DateType
(DeltaTable.createIfNotExists(spark)
    .tableName("default.covid_nyt_by_date")
    .property("description", "table with default partitions")
    .property("catalog.team_name", "dldg_authors")
    .property("catalog.engineering.comms.slack",
	"https://delta-users.slack.com/archives/CG9LR6LN4")
    .property("catalog.engineering.comms.email","dldg_authors@gmail.com")
    .property("catalog.table.classification","all-access")
    .addColumn("date", DateType(), nullable=False)
    .addColumn("county", "STRING")
    .addColumn("state", "STRING")
    .addColumn("fips", "INT")
    .addColumn("cases", "INT")
    .addColumn("deaths", "INT")
    .partitionedBy("date")
    .execute())

In [ ]:
# spark.sql("drop table default.covid_nyt_by_date")

In [ ]:
# Use our non-partitioned source table to populate our partitioned table (automatically)
(
    spark
    .table("default.covid_nyt")
    .write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "false")
    .saveAsTable("default.covid_nyt_by_date")
)

## Viewing the Partition Metadata of our Tables

In [166]:
spark.sql("describe extended default.covid_nyt_by_date").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                date|                date|       |
|              county|              string|       |
|               state|              string|       |
|                fips|                 int|       |
|               cases|                 int|       |
|              deaths|                 int|       |
|                    |                    |       |
|      # Partitioning|                    |       |
|              Part 0|                date|       |
|                    |                    |       |
|# Detailed Table ...|                    |       |
|                Name|default.covid_nyt...|       |
|            Location|file:/opt/spark/w...|       |
|            Provider|               delta|       |
|               Owner|              NBuser|       |
|    Table Properties|[catalog.engineer...|       |
+-----------

In [186]:
# view the table metadata as a json blob

DeltaTable.forName(spark, "default.covid_nyt_by_date").detail().toJSON().collect()[0]

'{"format":"delta","id":"8c57bc67-369f-4c84-a63e-38b8ac19bdf2","name":"default.covid_nyt_by_date","location":"file:/opt/spark/work-dir/ch6/spark-warehouse/covid_nyt_by_date","createdAt":"2023-06-08T05:35:00.072Z","lastModified":"2023-06-08T05:50:45.241Z","partitionColumns":["date"],"numFiles":423,"sizeInBytes":17660304,"properties":{"description":"table with default partitions","catalog.table.classification":"all-access","catalog.engineering.comms.email":"dldg_authors@gmail.com","catalog.team_name":"dldg_authors","catalog.engineering.comms.slack":"https://delta-users.slack.com/archives/CG9LR6LN4"},"minReaderVersion":1,"minWriterVersion":2,"tableFeatures":["appendOnly","invariants"]}'

# Create Bronze and Silver Databases

In [ ]:
spark.sql("show databases;").show()

In [ ]:
# We need to first create two databases (schemas) in our Hive metastore, or Unity Catalog.
# If using Unity Catalog, you can prefix <catalog>.<schema>.<table>
# With Hive, you can only use <schema>.<table>

spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

## COPY (CLONE) Tables between Databases (Schemas)
> We will be copying `default.covid_nyt_by_date` using DEEP CLONE into `bronze.covid_nyt_by_date` and `silver.covid_nyt_by_date`
> This functionality is available in the Databricks runtime as [CLONE](https://docs.databricks.com/delta/clone.html). Cloning is on the [roadmap](https://github.com/delta-io/delta/issues/1307) but not currently in the OSS Delta project.

In [10]:
from delta.tables import DeltaTable

# slim version of https://github.com/MrPowers/mack/blob/main/mack/__init__.py#L288
def copy_table(delta_table: DeltaTable, target_table: str):
    details = (
        delta_table
        .detail()
        .select("partitionColumns", "properties")
        .collect()[0]
    )
    (
        table_to_copy.toDF().write.format("delta")
        .partitionBy(details["partitionColumns"])
        .options(**details["properties"])
        .saveAsTable(target_table)
    )


In [ ]:
# copy the default table and write into both bronze and silver
table_to_copy = DeltaTable.forName(spark, "default.covid_nyt_by_date")
bronze_table = "bronze.covid_nyt_by_date"
silver_table = "silver.covid_nyt_by_date"

copy_table(table_to_copy, bronze_table)
copy_table(table_to_copy, silver_table)

In [16]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
|   silver|
+---------+



In [23]:
## Remove a partition from the silver table so we can repair the table with our bronze table
silver_dt = DeltaTable.forName(spark, "silver.covid_nyt_by_date")
silver_dt.delete(col("date") == "2021-02-17")

# Note: (if you delete, and then immediately vacuum, you will not be able to restore your table)
# vacuum to remove the physical data from the table
#spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")
#silver_dt.vacuum(retentionHours=0)
#spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","true")

## Using ReplaceWhere to do Conditional Repairs

## Restoring Tables to a Prior Version



In [34]:
dt = DeltaTable.forName(spark, "silver.covid_nyt_by_date")
dt.history(10).select("version", "timestamp", "operation").show()
dt.restoreToVersion(0)

+-------+--------------------+--------------------+
|version|           timestamp|           operation|
+-------+--------------------+--------------------+
|      4|2023-06-09 19:26:...|               WRITE|
|      3|2023-06-09 19:17:...|          VACUUM END|
|      2|2023-06-09 19:17:...|        VACUUM START|
|      1|2023-06-09 19:11:...|              DELETE|
|      0|2023-06-09 19:04:...|CREATE TABLE AS S...|
+-------+--------------------+--------------------+



## Cleaning up our Delta Tables using Vacuum

In [145]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")
DeltaTable.forName(spark, "default.nonoptimal_covid_nyt").vacuum(retentionHours=0)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","true")

Deleted 9000 files and directories in a total of 1 directories.


In [189]:
spark.sql("select distinct(date) as date from default.covid_nyt_by_date order by date desc").show(200)

+----------+
|      date|
+----------+
|2021-03-11|
|2021-03-10|
|2021-03-09|
|2021-03-08|
|2021-03-07|
|2021-03-06|
|2021-03-05|
|2021-03-04|
|2021-03-03|
|2021-03-02|
|2021-03-01|
|2021-02-28|
|2021-02-27|
|2021-02-26|
|2021-02-25|
|2021-02-24|
|2021-02-23|
|2021-02-22|
|2021-02-21|
|2021-02-20|
|2021-02-19|
|2021-02-18|
|2021-02-17|
|2021-02-16|
|2021-02-15|
|2021-02-14|
|2021-02-13|
|2021-02-12|
|2021-02-11|
|2021-02-10|
|2021-02-09|
|2021-02-08|
|2021-02-07|
|2021-02-06|
|2021-02-05|
|2021-02-04|
|2021-02-03|
|2021-02-02|
|2021-02-01|
|2021-01-31|
|2021-01-30|
|2021-01-29|
|2021-01-28|
|2021-01-27|
|2021-01-26|
|2021-01-25|
|2021-01-24|
|2021-01-23|
|2021-01-22|
|2021-01-21|
|2021-01-20|
|2021-01-19|
|2021-01-18|
|2021-01-17|
|2021-01-16|
|2021-01-15|
|2021-01-14|
|2021-01-13|
|2021-01-12|
|2021-01-11|
|2021-01-10|
|2021-01-09|
|2021-01-08|
|2021-01-07|
|2021-01-06|
|2021-01-05|
|2021-01-04|
|2021-01-03|
|2021-01-02|
|2021-01-01|
|2020-12-31|
|2020-12-30|
|2020-12-29|
|2020-12-28|

In [1]:
spark.sql("select count(distinct(date)) from default.covid_nyt_by_date").show()

23/06/08 23:57:12 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/08 23:57:12 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/06/08 23:57:13 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/06/08 23:57:13 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.17.0.2
23/06/08 23:57:13 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/06/08 23:57:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+
|count(DISTINCT date)|
+--------------------+
|                 416|
+--------------------+

